# Fine tune ResNet50 for generated dataset

In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision

In [2]:
import os
from PIL import Image
from tqdm import tqdm
import numpy as np

In [3]:
trainData_Dir = "D:\\project\\dataset\\gen_dataset"

# Define the transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomRotation(degrees=15),
    transforms.Grayscale(num_output_channels=3),  # Resize images to a fixed size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

dataset = datasets.ImageFolder(trainData_Dir, transform=train_transform)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
t , v = random_split(dataset, [train_size, val_size])

batch_size = 10
shuffle_data = True




train_loader = DataLoader(t, batch_size=batch_size, shuffle=shuffle_data)
val_loader = DataLoader(v, batch_size=batch_size, shuffle=False)



In [6]:
dataset[0][0]

tensor([[[-0.5767, -0.6281, -0.6452,  ...,  0.6049,  0.6049,  0.5878],
         [-0.5767, -0.6109, -0.6452,  ...,  0.6049,  0.6049,  0.5878],
         [-0.5767, -0.5938, -0.6109,  ...,  0.6049,  0.6049,  0.6049],
         ...,
         [ 1.2728,  1.2728,  1.2728,  ...,  0.8789,  0.8789,  0.8789],
         [ 1.2728,  1.2728,  1.2728,  ...,  0.8789,  0.8789,  0.8789],
         [ 1.2728,  1.2728,  1.2728,  ...,  0.8789,  0.8789,  0.8789]],

        [[-0.4601, -0.5126, -0.5301,  ...,  0.7479,  0.7479,  0.7304],
         [-0.4601, -0.4951, -0.5301,  ...,  0.7479,  0.7479,  0.7304],
         [-0.4601, -0.4776, -0.4951,  ...,  0.7479,  0.7479,  0.7479],
         ...,
         [ 1.4307,  1.4307,  1.4307,  ...,  1.0280,  1.0280,  1.0280],
         [ 1.4307,  1.4307,  1.4307,  ...,  1.0280,  1.0280,  1.0280],
         [ 1.4307,  1.4307,  1.4307,  ...,  1.0280,  1.0280,  1.0280]],

        [[-0.2358, -0.2881, -0.3055,  ...,  0.9668,  0.9668,  0.9494],
         [-0.2358, -0.2707, -0.3055,  ...,  0

In [6]:
mapper = {}
mapper_ = {}
for i, dir_name in enumerate(os.listdir(trainData_Dir)):
    mapper[i] = dir_name
    mapper_[dir_name] = i

In [7]:
mapper

{0: 'a',
 1: 'b',
 2: 'c',
 3: 'd',
 4: 'e',
 5: 'f',
 6: 'g',
 7: 'h',
 8: 'i',
 9: 'j',
 10: 'k',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'q',
 17: 'r',
 18: 's',
 19: 't',
 20: 'u',
 21: 'v',
 22: 'w',
 23: 'x',
 24: 'y',
 25: 'z'}

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
def validate(model, dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    val_running_errors = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
            #print(data[0].shape)
            data, labels = data[0].to(device), data[1].to(device)

            outputs = model(data)
            #print(outputs)
            #print(labels)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()* data.size(0)
            _, preds = torch.max(outputs.data, 1)
            val_running_correct += torch.sum(preds == labels)
            val_running_errors += torch.sum(preds != labels)

        val_loss = val_running_loss/len(dataloader.dataset)
        val_accuracy = val_running_correct/len(dataloader.dataset)
        val_errore = val_running_errors/len(dataloader.dataset)
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy}')
        
        return val_loss, val_accuracy, val_errore
# train part
def train(model,optimizer, dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    train_running_errors = 0
    for i, data in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset) /dataloader.batch_size)):
        #print(data[1])
        data, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(data)
        #print(outputs)
        #print(labels.dtype)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs.data, 1)
        train_running_errors += torch.sum(preds != labels)
        train_running_correct += torch.sum(preds == labels)
        train_running_loss += loss.item()* data.size(0)
        loss.backward()
        optimizer.step()
        
    train_loss = train_running_loss/len(dataloader.dataset)
    train_accuracy = train_running_correct/len(dataloader.dataset)
    train_errore = train_running_errors/len(dataloader.dataset)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy}")
    return train_loss, train_accuracy, train_errore
 #  main 
def model_train(model, optimizer, train_dataloader ,test_dataloader, epochs):
  err_history = {"train" : [], "val" : []}
  acc_history =  {"train" : [], "val" : []}
  loss_history = {"train" : [], "val" : []}
  for epoch in range(epochs):
      print(f"Epoch {epoch+1} of {epochs}")
      phase = "train"
      print(phase)
      loss, accuracy, error = train(model, optimizer, train_dataloader)
      err_history[phase].append(error)
      acc_history[phase].append(accuracy)
      loss_history[phase].append(loss)
      phase = "val"
      print(phase)
      loss, accuracy, error = validate(model, test_dataloader)
      err_history[phase].append(error)
      acc_history[phase].append(accuracy)
      loss_history[phase].append(loss)
  save_model(model)
  return err_history, acc_history, loss_history

In [11]:
checkpoint_path = "D:/project/asl/checkpoint_data_gen.pt"

def save_model(model):
    torch.save(model.state_dict(), checkpoint_path)

In [12]:
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from torchvision.models.detection.mask_rcnn import MaskRCNN
m = resnet50()
train_nodes, eval_nodes = get_graph_node_names(resnet50())
return_nodes = {
    'layer1': 'layer1',
    'layer2': 'layer2',
    'layer3': 'layer3',
    'layer4': 'layer4',
}
create_feature_extractor(m, return_nodes=return_nodes)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Module(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=Fal

In [13]:
class Resnet50_Fine(torch.nn.Module):
    def __init__(self,num_classes):
        super(Resnet50_Fine, self).__init__()

        self.pretrained = resnet50(pretrained=True)

        num_ftrs = self.pretrained.fc.in_features
        self.pretrained.fc = nn.Linear(num_ftrs, num_classes)
    def forward(self, x):
        x = self.pretrained(x)
        return x

model = Resnet50_Fine(num_classes=26)

c:\Users\Yuriy\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Yuriy\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
opt_myCNN = optim.Adam(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()
num_epochs=2

In [15]:
myCNN_err_history, myCNN_acc_history , myCNN_loss_history = model_train(model, opt_myCNN, train_loader, val_loader ,num_epochs )

Epoch 1 of 2
train


100%|██████████| 351/351 [01:13<00:00,  4.81it/s]


Train Loss: 0.6926, Train Acc: 0.8487179279327393
val


100%|██████████| 39/39 [00:03<00:00, 10.79it/s]


Val Loss: 0.0434, Val Acc: 0.9948718547821045
Epoch 2 of 2
train


100%|██████████| 351/351 [01:06<00:00,  5.28it/s]


Train Loss: 0.0838, Train Acc: 0.9849002361297607
val


100%|██████████| 39/39 [00:03<00:00, 11.22it/s]


Val Loss: 0.0117, Val Acc: 1.0


In [13]:
def validate_(model, dataloader):
    model.eval()
    print("1")
    val_running_loss = 0.0
    val_running_correct = 0
    val_running_errors = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(dataloader.dataset)/dataloader.batch_size)):
            #print(data[0].shape)
            data, labels = data[0].to(device), data[1].to(device)

            outputs = model(data)
            print(labels)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item()* data.size(0)
            _, preds = torch.max(outputs.data, 1)
            print(preds)
        
        return outputs, labels, preds

In [14]:
testData_Dir = "D:\project\dataset\ASL_Dataset\Test"

# Define the transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

test = datasets.ImageFolder(testData_Dir, transform=train_transform)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [15]:
validate_(model, test_loader)

1


  9%|▉         | 1/11 [00:00<00:01,  7.09it/s]

tensor([0, 0, 0, 0, 1, 1, 1, 1, 2, 2], device='cuda:0')
tensor([16, 16, 18, 16,  1,  1,  1,  1,  5,  2], device='cuda:0')


 18%|█▊        | 2/11 [00:00<00:01,  6.89it/s]

tensor([2, 2, 3, 3, 3, 3, 4, 4, 4, 4], device='cuda:0')
tensor([ 5,  5, 20, 20, 20, 20, 13, 13, 13, 13], device='cuda:0')


...

tensor([20, 20, 20, 20, 21, 21, 21, 21, 22, 22], device='cuda:0')
tensor([ 9,  9,  9,  9, 11, 24, 24, 24, 20, 20], device='cuda:0')


 91%|█████████ | 10/11 [00:01<00:00,  7.26it/s]

tensor([22, 22, 23, 23, 23, 23, 24, 24, 24, 24], device='cuda:0')
tensor([20, 20, 22, 22, 22, 22, 22, 22, 22, 22], device='cuda:0')
tensor([25, 25, 25, 25, 26, 26, 26, 26, 27, 27], device='cuda:0')


 91%|█████████ | 10/11 [00:02<00:00,  4.63it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
